In [67]:
import pandas as pd
import bs4 as bs
import requests
import time
import numpy as np

In [68]:
df = pd.read_excel('./Fin_Res_Q3_22_23_12_Feb_2023_in_V1.xlsx')
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
urls = df["res_url"].tolist()


In [69]:
def get_data(url):
    resp = requests.get(url)
    soup = bs.BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table', {'class': 'mctable1'})
    data = []
    if table is None:
        return data
    for row in table.findAll('tr'):
        cols = row.findAll('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return data

In [70]:
outdf = pd.DataFrame(columns=["SYMB", " DEC '22_R", " DEC '22_V", " DEC '22_DEPS",
                                " DEC '22_Interest", " DEC '22_Gross NPA", " DEC '22_Net NPA", "LUU",
                                    "RES_CATG_1", "RES_CATG_2", "L_RES_DT", "DATA_AVAILABLE",
                                    "Last Available Data", "TIME STAMP", "RES_URL", "SECTOR"])

In [71]:
def fill_df(urls):
    for url in urls:
        data = get_data(url)
        if len(data) == 0:
            outdf.loc[urls.index(url), "RES_URL"] = df.loc[urls.index(url)+1, "res_url"]
            outdf.loc[urls.index(url), "DATA_AVAILABLE"] = "N"
            outdf.loc[urls.index(url), "TIME STAMP"] = time.strftime("%d/%m/%Y %H:%M:%S")
            outdf.loc[urls.index(url), "SECTOR"] = df.loc[urls.index(url)+1, "sectr"]
            outdf.loc[urls.index(url),'SYMB'] = df.loc[urls.index(url)+1, "symb"]
            outdf.loc[urls.index(url), "LUU"] = df.loc[urls.index(url)+1, "LUU"]
            outdf.loc[urls.index(url), "RES_CATG_1"] = df.loc[urls.index(url)+1, "res_catg1"]
            outdf.loc[urls.index(url), "RES_CATG_2"] = df.loc[urls.index(url)+1, "res_catg2"]
            outdf.loc[urls.index(url), "L_RES_DT"] = df.loc[urls.index(url)+1, "l_res_dt"]
            outdf.loc[urls.index(url), "Last Available Data"] = "NONE"
        elif len(data) > 0:
            dg = pd.DataFrame(data,index=None,columns=None)
            dg.columns = dg.iloc[0]
            dg = dg.drop(dg.index[0])
            dg = dg.iloc[:, :2]
            outdf.loc[urls.index(url),'SYMB'] = df.loc[urls.index(url)+1, "symb"]
            outdf.loc[urls.index(url), "LUU"] = df.loc[urls.index(url)+1, "LUU"]
            outdf.loc[urls.index(url), "RES_CATG_1"] = df.loc[urls.index(url)+1, "res_catg1"]
            outdf.loc[urls.index(url), "RES_CATG_2"] = df.loc[urls.index(url)+1, "res_catg2"]
            outdf.loc[urls.index(url), "L_RES_DT"] = df.loc[urls.index(url)+1, "l_res_dt"]
            outdf.loc[urls.index(url), "RES_URL"] = df.loc[urls.index(url)+1, "res_url"]
            outdf.loc[urls.index(url), "SECTOR"] = df.loc[urls.index(url)+1, "sectr"]
            outdf.loc[urls.index(url), " DEC '22_R"] = dg.loc[1][1]
            outdf.loc[urls.index(url), " DEC '22_V"] = dg.loc[28][1]
            outdf.loc[urls.index(url), " DEC '22_DEPS"] = dg.loc[37][1]
            outdf.loc[urls.index(url), " DEC '22_Interest"] = dg.loc[20][1]
            outdf.loc[urls.index(url), "DATA_AVAILABLE"] = "Y"
            outdf.loc[urls.index(url), "Last Available Data"] = dg.columns[1]
            outdf.loc[urls.index(url), "TIME STAMP"] = time.strftime("%d/%m/%Y %H:%M:%S")
            if outdf.loc[urls.index(url), "SECTOR"] == "Banks":
                outdf.loc[urls.index(url), " DEC '22_R"] = dg.loc[2][1]
                outdf.loc[urls.index(url), " DEC '22_V"] = dg.loc[20][1]
                outdf.loc[urls.index(url), " DEC '22_DEPS"] = dg.loc[30][1]
                outdf.loc[urls.index(url), " DEC '22_Interest"] = "0"
                outdf.loc[urls.index(url), " DEC '22_Gross NPA"] = dg.loc[35][1]
                outdf.loc[urls.index(url), " DEC '22_Net NPA"] = dg.loc[36][1]
                


    return outdf

In [72]:
output = fill_df(urls)
output.to_excel("output.xlsx", index=False)
output

ChunkedEncodingError: ("Connection broken: ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)", ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))